<a href="https://colab.research.google.com/github/jbloewencolon/The-Void/blob/main/Psychedelics_and_The_Void.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk

import pandas as pd
import re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import bigrams, trigrams
from nltk.probability import FreqDist
from collections import Counter
import string
import os
import pickle
from nltk.corpus import stopwords

# Download NLTK stop words if not already downloaded
nltk.download('stopwords')

# Download NLTK data if not already downloaded
nltk.download('punkt')

# Download the VADER lexicon (if not already downloaded)
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Psychedelic-Trip-Report-Generator/Data/processed.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67875 entries, 0 to 67874
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Unnamed: 0        67875 non-null  int64 
 1   drug              67875 non-null  object
 2   dosage            66103 non-null  object
 3   delivery          65447 non-null  object
 4   weight            67875 non-null  int64 
 5   gender            67875 non-null  object
 6   report            67875 non-null  object
 7   processed_report  67875 non-null  object
 8   mixed             67875 non-null  int64 
 9   drug_category     67875 non-null  object
dtypes: int64(3), object(7)
memory usage: 5.2+ MB


In [ ]:
def find_and_print_void_sentences(dataframe):
    # Iterate through the "report" column
    for index, row in dataframe.iterrows():
        report_text = row['report']

        # Split the report text into sentences using regular expressions
        sentences = re.split(r'(?<=[.!?])\s+', report_text)

        # Check each sentence for the word "Void"
        for sentence in sentences:
            if 'Void' in sentence:
                print(f"Report Index: {index}, Sentence: {sentence}\n")

# Assuming your DataFrame is named df
find_and_print_void_sentences(df)

In [ ]:
def analyze_sentiment_with_void(dataframe):
    # Initialize the Sentiment Intensity Analyzer
    sia = SentimentIntensityAnalyzer()

    # Iterate through the "report" column
    for index, row in dataframe.iterrows():
        report_text = row['processed_report']

        # Split the report text into sentences using regular expressions
        sentences = re.split(r'(?<=[.!?])\s+', report_text)

        # Check each sentence for the word "Void"
        for sentence in sentences:
            if 'Void' in sentence:
                sentiment_scores = sia.polarity_scores(sentence)
                print(f"Report Index: {index}, Sentence: {sentence}")
                print("Sentiment Scores:", sentiment_scores)
                print("\n")

# Assuming your DataFrame is named df
analyze_sentiment_with_void(df)


In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
import re

def categorize_sentiment_reports(dataframe):
    # Initialize the Sentiment Intensity Analyzer
    sia = SentimentIntensityAnalyzer()

    # Initialize counters for each sentiment category and for mixed and not mixed reports
    positive_count = 0
    neutral_count = 0
    negative_count = 0
    mixed_positive_count = 0
    mixed_neutral_count = 0
    mixed_negative_count = 0
    not_mixed_positive_count = 0
    not_mixed_neutral_count = 0
    not_mixed_negative_count = 0

    # Iterate through the DataFrame
    for index, row in dataframe.iterrows():
        report_text = row['report']
        is_mixed = row['mixed']

        # Split the report text into sentences using regular expressions
        sentences = re.split(r'(?<=[.!?])\s+', report_text)

        # Initialize sentiment category counts for the current report
        current_positive_count = 0
        current_neutral_count = 0
        current_negative_count = 0

        # Check each sentence for the word "Void" and categorize by sentiment
        for sentence in sentences:
            if 'Void' in sentence:
                sentiment_scores = sia.polarity_scores(sentence)
                compound_score = sentiment_scores['compound']

                if compound_score >= 0.05:
                    current_positive_count += 1
                elif compound_score <= -0.05:
                    current_negative_count += 1
                else:
                    current_neutral_count += 1

        # Update the overall sentiment category counts
        positive_count += current_positive_count
        neutral_count += current_neutral_count
        negative_count += current_negative_count

        # Categorize mixed and not mixed reports
        if is_mixed:
            mixed_positive_count += current_positive_count
            mixed_neutral_count += current_neutral_count
            mixed_negative_count += current_negative_count
        else:
            not_mixed_positive_count += current_positive_count
            not_mixed_neutral_count += current_neutral_count
            not_mixed_negative_count += current_negative_count

    # Print the counts for each sentiment category and mixed vs. not mixed reports
    print("Positive Reports:", positive_count)
    print("Neutral Reports:", neutral_count)
    print("Negative Reports:", negative_count)
    print("Mixed Positive Reports:", mixed_positive_count)
    print("Mixed Neutral Reports:", mixed_neutral_count)
    print("Mixed Negative Reports:", mixed_negative_count)
    print("Not Mixed Positive Reports:", not_mixed_positive_count)
    print("Not Mixed Neutral Reports:", not_mixed_neutral_count)
    print("Not Mixed Negative Reports:", not_mixed_negative_count)

# Assuming your DataFrame is named df
categorize_sentiment_reports(df)

Positive Reports: 102
Neutral Reports: 65
Negative Reports: 37


In [ ]:
# Function to clean and tokenize text, removing stop words
def clean_and_tokenize(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # Remove words with less than 2 characters and any punctuation, numbers, or symbols
    tokens = [word for word in tokens if len(word) >= 2 and word.isalpha()]

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.lower() not in stop_words]

    return tokens
# Function to save tokenized data to a file
def save_tokenized_data(tokens, filename):
    with open(filename, 'wb') as file:
        pickle.dump(tokens, file)

# Function to load tokenized data from a file
def load_tokenized_data(filename):
    with open(filename, 'rb') as file:
        tokens = pickle.load(file)
    return tokens

# Check if tokenized data file exists, if not, tokenize and save
tokenized_data_filename = 'tokenized_data.pkl'
if os.path.exists(tokenized_data_filename):
    processed_report_tokens = load_tokenized_data(tokenized_data_filename)
else:
    processed_report_tokens = []
    for index, row in df.iterrows():
        tokens = clean_and_tokenize(row['report'])
        processed_report_tokens.append(tokens)

    # Save tokenized data to a file
    save_tokenized_data(processed_report_tokens, tokenized_data_filename)


In [ ]:
def find_top_word_bigrams_and_trigrams(dataframe):
    # Create empty lists to store word bigrams and trigrams
    void_word_bigrams = []
    void_word_trigrams = []

    # Iterate through the "processed_report" column
    for index, row in dataframe.iterrows():
        processed_report_tokens = clean_and_tokenize(row['report'])

        # Check if "void" is present in the tokens
        if 'void' in processed_report_tokens:
            void_index = processed_report_tokens.index('void')

            # Generate word bigrams and trigrams around "void"
            void_word_bigrams.extend(list(bigrams(processed_report_tokens))[max(0, void_index - 1):min(void_index + 2, len(processed_report_tokens))])
            void_word_trigrams.extend(list(trigrams(processed_report_tokens))[max(0, void_index - 1):min(void_index + 3, len(processed_report_tokens))])

    # Calculate the most common word bigrams and trigrams
    common_void_word_bigrams = FreqDist(void_word_bigrams).most_common(50)
    common_void_word_trigrams = FreqDist(void_word_trigrams).most_common(50)

    # Print the results
    print("Top 50 Word Bigrams around 'void':")
    for bigram, count in common_void_word_bigrams:
        print(f"{bigram}: {count}")

    print("\nTop 50 Word Trigrams around 'void':")
    for trigram, count in common_void_word_trigrams:
        print(f"{trigram}: {count}")

# Assuming your DataFrame is named df
find_top_word_bigrams_and_trigrams(df)

Top 50 Word Bigrams around 'void':
('black', 'void'): 160
('void', 'space'): 43
('empty', 'void'): 43
('dark', 'void'): 39
('infinite', 'void'): 33
('void', 'felt'): 33
('fill', 'void'): 30
('sort', 'void'): 30
('floating', 'void'): 29
('great', 'void'): 28
('void', 'could'): 28
('void', 'like'): 27
('like', 'void'): 27
('back', 'void'): 26
('endless', 'void'): 25
('white', 'void'): 23
('void', 'place'): 22
('void', 'filled'): 22
('void', 'behind'): 21
('vast', 'void'): 20
('void', 'time'): 20
('experience', 'void'): 18
('void', 'nothingness'): 17
('completely', 'void'): 17
('void', 'seems'): 17
('blackness', 'void'): 16
('void', 'would'): 15
('complete', 'void'): 15
('void', 'universe'): 14
('void', 'beyond'): 14
('void', 'feel'): 14
('void', 'left'): 14
('entered', 'void'): 14
('swirling', 'void'): 14
('void', 'center'): 13
('null', 'void'): 13
('void', 'background'): 13
('nothing', 'void'): 13
('void', 'one'): 13
('void', 'thought'): 13
('void', 'friend'): 13
('time', 'void'): 13
('

In [ ]:
max_results_to_print = 50  # Set your desired limit
printed_count = 0

for index, report_text in matching_reports:
    if printed_count >= max_results_to_print:
        break

    print(f"Report Index: {index}")

    # Split the report text into sentences using regular expressions
    sentences = re.split(r'(?<=[.!?])\s+', report_text)

    # Check each sentence for the presence of 'void' or any single keyword
    for sentence in sentences:
        words = sentence.split()  # Split sentence into words
        if 'void' in words:
            print(f"Sentence (Contains 'void'): {sentence}\n")
        else:
            for keyword in keywords_to_search:
                if keyword in words:
                    print(f"Sentence (Contains '{keyword}'): {sentence}\n")

    printed_count += 1

def search_reports_for_keywords(dataframe, keywords, min_keyword_count=2):
    # Initialize an empty list to store matching reports
    matching_reports = []

    # Iterate through the DataFrame
    for index, row in dataframe.iterrows():
        report_text = row['report']

        # Check if the report contains the word "void"
        if 'void' in report_text:
            # Count how many of the specified keywords are in the report
            keyword_count = sum(keyword in report_text for keyword in keywords)

            # Check if the count of matching keywords is at least min_keyword_count
            if keyword_count >= min_keyword_count:
                matching_reports.append((index, report_text))

    return matching_reports

# List of keywords to search for
keywords_to_search = ["null", "empty", "missing", "error", "absence", "invalid", "blank", "unspecified", "deleted", "unavailable"]

# Assuming your DataFrame is named df
matching_reports = search_reports_for_keywords(df, keywords_to_search, min_keyword_count=2)

# Print matching reports
for index, report_text in matching_reports:
    print(f"Report Index: {index}, Report Text: {report_text}\n")


output_filename = 'matching_reports.txt'

with open(output_filename, 'w') as output_file:
    for index, report_text in matching_reports:
        output_file.write(f"Report Index: {index}, Report Text: {report_text}\n")

print(f"Matching reports have been saved to {output_filename}")



Report Index: 21
Sentence (Contains 'missing'): My brother stuck his tongue out to reveal the two missing tabs resting on top.

Sentence (Contains 'empty'): I found this teddy bear I use to play with that I gave a personality to and made voices for, but it just felt like an empty teddy bear with no presence at all.+5:00 I would say I had passed the hardships of the trip at this point.

Report Index: 62
Sentence (Contains 'empty'): The substances were ingested only orally, with powders either dissolved in a beverage or swallowed in a gel cap, typically on a well digested or empty stomach.IntroductionHaving experimented with these chemicals relatively extensively, I summarize here my experiences and present some of the conclusions---impressions, perhaps---that have emerged during my protracted period of reflection.For general comparison purposes, typical trip durations were as follows:2C-I:  7-9 hours up; 40-60 minute come-up2C-E:  8-10 hours up; 110-160 minute come-up, 130 minutes avera

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
# Assuming your DataFrame is named df

# Filter the DataFrame to select only rows with "void" in the report
void_reports = df[df['report'].str.contains('void', case=False, na=False)]

# Select the demographic columns of interest
demographic_data = void_reports[['gender', 'drug', 'mixed', 'weight']]

# Display the demographic data for reports containing "void"
print(demographic_data)


In [ ]:
# Filter the DataFrame to select only rows with "void" in the report
void_reports = df[df['report'].str.contains('void', case=False, na=False)]

# Calculate the total count of males and females
gender_counts = void_reports['gender'].value_counts()

# Calculate the average weight
average_weight = void_reports['weight'].mean()

# Calculate the drug types that produced the word "void" more frequently
drug_counts = void_reports['drug'].value_counts()

# Calculate the count of reports that are mixed and not mixed
mixed_counts = void_reports['mixed'].value_counts()

# Display the results
print(f"Total Count of Males: {gender_counts.get('male', 0)}")
print(f"Total Count of Females: {gender_counts.get('female', 0)}")
print(f"Average Weight: {average_weight:.2f} grams")
print("Drug Types Producing 'Void' More Frequently:")
print(drug_counts)
print(f"Count of Mixed Reports: {mixed_counts.get(1, 0)}")
print(f"Count of Not Mixed Reports: {mixed_counts.get(0, 0)}")


Total Count of Males: 6326
Total Count of Females: 895
Average Weight: 161.97 grams
Drug Types Producing 'Void' More Frequently:
cannabis                644
lsd                     554
salvia divinorum        482
mushrooms               408
mdma                    388
                       ... 
vitamin b-6               1
methcathinone             1
eth-lad                   1
pharms - propranolol      1
turbina corymbosa         1
Name: drug, Length: 416, dtype: int64
Count of Mixed Reports: 4651
Count of Not Mixed Reports: 2984
